In [1]:
# load packages
import numpy as np 
import scipy 
import pandas as pd
import os
import matplotlib.pyplot as plt #import matplotlib as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns #import mat73
import pickle as pkl
from datetime import datetime
import mne
from mne.viz import plot_topomap
from mne.viz import set_3d_title, set_3d_view
sns.set_theme() # set the plotting atmosphere

### Import all files that will be sorted and exported into another .mat file

In [2]:
# Main Directory of processed file from MatLab
#MainDir = 'D:\\AttnXV3_analysis\\RCA_F1\\AvgCRFs\\' # set dir
MainDir = 'C:\\plimon\\LTP_analysis\\eegMatFiles\\MAT' # set dir
os.chdir(MainDir) # change old dir, to this dir
dataFileNames = os.listdir(MainDir) # list files in dir
print(f'Total Files Avilable: {len(dataFileNames)}')
##############################################
FileN = dataFileNames[34]# choose one                        
file_path1 = os.path.join(MainDir, FileN) # join paths and prep 2 load
print('Current WD:',file_path1) # does path exist ... ?
print('Does File #1 Exist?',os.path.exists(file_path1)) # yes or no

Total Files Avilable: 50
Current WD: C:\plimon\LTP_analysis\eegMatFiles\MAT\nl-2697_attnL_20231120_1601_RLS.mat
Does File #1 Exist? True


#### Remove # to import all files and then just use this (saves compu time)

In [3]:
# # small script to import data + filenames for 1 condition 
# SignalDataDict={} # fill with subjects signal data
# FileNamesIndicies = []
# for Fname in range(len(dataFileNames)):
#     dataFileIn = dataFileNames[Fname]
#     filePath =  os.path.join(MainDir, dataFileIn) # join paths and prep 2 load
#     mat_data = scipy.io.loadmat(filePath) # import data file
#     MainData = mat_data['signalData'][0][0] # condition and 
#     #print(MainData.shape)
#     SignalDataDict[Fname] = MainData
#     FileNamesIndicies.append(dataFileIn)
    
# FileNamesIndicies = np.array(FileNamesIndicies) # make list into array 
# FileName = [FileNamesIndicies.replace('_','-') for FileNamesIndicies in FileNamesIndicies] # uniform fnames to make bools for attnl and attnR 
# DataOut = {}

# DataOut['RawData'] = SignalDataDict
# DataOut['SubjNames'] = FileName
# SaveDataDir = 'C:\\plimon\\LTP_analysis\\eegMatFiles\\AllMat'
# print(os.path.exists(SaveDataDir)) # does pth exist :p

# dnt = datetime.now() # add date and time bc im wreckless when saving ..
# fdnt = dnt.strftime("%Y%m%d_%H%M%S") # set the above as a string ...
# FileOutName = 'AllSubj_MatFiles_C1'
# FileN = f'{FileOutName}_{fdnt}.pkl' 
# NewFileNPath = os.path.join(SaveDataDir,FileN)
# print('Full New File Dir: ', NewFileNPath)

# saveFile = 'y'
# if saveFile == 'y':
#  with open(NewFileNPath, 'wb') as file:
#     pkl.dump(DataOut, file, protocol=pkl.HIGHEST_PROTOCOL)
#     print('MAT Data For All Subjs Saved! :))')
# else:
#     print('Did Not Save File! Change file name before switching to y!')

#### Print Keys in original data

In [4]:
mat_data = scipy.io.loadmat(file_path1) # import data file
print((mat_data.keys()))

dict_keys(['__header__', '__version__', '__globals__', 'info', 'signalData', 'noise1', 'noise2'])


#### Index certain keys for analysis and conditions during expt

In [5]:
# find basic data parms 
MainData = mat_data['signalData']
noise1 = mat_data['noise1'] # 6 x 1 
noise2 = mat_data['noise2']
inds = mat_data['info'][0][0]
ExpConds = int(MainData.shape[0])
print(f'Data file contains data for {ExpConds} different conditions')

Data file contains data for 6 different conditions


### Select Channels of Interest to Visualize Later

In [6]:
elecLeftLateral=[51,52,60,58,59,64,65,68,69]
elecLeftMedial=[72,75,81,70,71,74]
elecRightMedial=[72,75,81,76,83,82]
elecMedial=[72,75,81,70,71,74,76,83,82]
elecRightLateral=[97,92,85,96,91,95,90,94,88]
extraChans = np.arange(50,100,1) #
extra = [107,101,47,42,37,31,102,46,41,45,49,103,108,113,108,99]

AnalysisChans = [elecLeftLateral,elecLeftMedial,elecRightMedial,elecMedial,elecRightLateral,extraChans,extra]
ExportChans = np.unique(np.sort(np.concatenate(AnalysisChans))) # Combine array for chans of interest
ExportChans  = ExportChans - 1

# ExportChans = [71,74]
print(f'Channels to be visualized: {ExportChans}')

Channels to be visualized: [ 30  36  40  41  44  45  46  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98 100 101 102 106 107 112]


#### Index Condition Data, Noise 1 and Noise 2

In [7]:
# save signal data
FreqDataPerCo = {}
for co in range(ExpConds):
    #print(MainData[co][0].shape)
    FreqDataPerCo[co] = MainData[co][0]
# noise data 1 
Noise1DataPerCo = {}
for co in range(ExpConds):
    #print(noise1[co][0].shape)
    Noise1DataPerCo[co] = noise1[co][0]
# noise data 2
Noise2DataPerCo = {}
for co in range(ExpConds):
    #print(noise2[co][0].shape)
    Noise2DataPerCo[co] = noise2[co][0]

#### Indicies To Extract

In [8]:
co = 0 # pre and post induction ltp
HarmInd = np.ravel(mat_data['info'][0][0] [0][co][0])
BinInd = np.ravel(mat_data['info'][0][0] [1][co][0])
#HarmonLabels = mat_data['info'][0][0][2][co][0]
AvgAct = mat_data['info'][0][0] [3][co] [0] # returns an array with 6 values ...
NumCondInd = mat_data['info'][0][0][4][co][0] # bin index, includes preludes
HemiFieldLabels = mat_data['info'][0][0] [2][co][0] # contains harmonic and hemifield data
hemiFieldnHarmonicInd = mat_data['info'][0][0] [2][co] [0]
#########In case some nested arrays need to be extracted or explored
# co = 0
# ind = 2
# inds = mat_data['info'][0][0] [ind][co] [0] # avoid prelude bind which == 0 actually 
# #print(inds)
# for innerlist in inds:
#     for item in innerlist:
#        print(item)

#### Get and Set Boundaries for Data Sorting 

In [9]:
NumHarms,NumHarmCount = np.unique(HarmInd, return_counts = True)
print(f'Hamronics:  {NumHarms,NumHarmCount}')
NumBins,NumBinCount = np.unique(BinInd, return_counts = True)
print(f'Bin Indicies:  {NumBins,NumBinCount}')
Bins = int(len(NumBins)-1)
print(f'Total Bins #{Bins}')

Hamronics:  (array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8), array([7, 7, 7, 7, 7, 7, 7, 7, 7], dtype=int64))
Bin Indicies:  (array([0, 1, 2, 3, 4, 5, 6], dtype=uint8), array([9, 9, 9, 9, 9, 9, 9], dtype=int64))
Total Bins #6


In [ ]:
SortDataIn =  FreqDataPerCo[0] # 112 x 128 x 78
[NumRows, NumChans, NumTrials] = SortDataIn.shape # get dims of data to use later ..
print(f'DataVals, Channels , Total Trials: {SortDataIn.shape}')
real_imag_bound = int((NumRows/2))
print(f'Data real-imag bound is at row: #{real_imag_bound}')
chanInd = np.arange(0,NumChans,1)
print(f'total channels: {chanInd.shape}')
prepost_bound = int(NumTrials/2)
HemiBound = int(int(len(HemiFieldLabels))/2)
print(f'Data will be split at the 4th row when called for hemifield segmentation')

#### Make Bool mask for MNE Topoplots

In [ ]:
ocm = np.zeros((NumChans), dtype = bool) # mask of len nchans
ocm[ExportChans] = True # set good chans to true 

#### Combine Real and Imaginary Components of data

In [ ]:
# Reshape data into real and imaginary data side by side
dIn = FreqDataPerCo[co].reshape(real_imag_bound,2,NumChans,NumTrials) # 56 x 2 x 128 x 78
n1 = Noise1DataPerCo[co].reshape(real_imag_bound,2,NumChans,NumTrials)
n2 = Noise2DataPerCo[co].reshape(real_imag_bound,2,NumChans,NumTrials)
# Clean Data
# remove 0s -> nans
dIn[dIn == 0] = np.nan
n1[n1 == 0] = np.nan
n2[n2 == 0] = np.nan
# empty dataframes
DataOut = np.zeros((real_imag_bound,NumChans,NumTrials))
n1Out = np.zeros_like(DataOut)
n2Out = np.zeros_like(DataOut)
# combine real and imaginary compnents for signal data, noise 1 and noise 2 
for val in range(real_imag_bound):
    DataOut[val,:,:] = np.hypot(dIn[val,0,:,:],dIn[val,1,:,:]) # (56,128, 78)
    n1Out[val,:,:] = np.hypot(n1[val,0,:,:],n1[val,1,:,:]) # (56, 128, 78)
    n2Out[val,:,:] = np.hypot(n2[val,0,:,:],n2[val,1,:,:]) # (56, 128, 78)
print(DataOut.shape)

#### Get Data Per Bin and Store in Dict, Remove Prelude Bins

In [ ]:
dataPerBin = {}
for i in range(len(NumBins)):
    if i > 0:
        bin_arr = np.zeros_like((BinInd))
        bin_arr[BinInd == i] = 1 # changr bool ind to get bin specific data
        dataPerBin[i-1] = DataOut[bin_arr == 1,:,:]

## **Check Point #1** : Make sure 1F1 harmonic data is **NOT** included in analysis

In [ ]:
colDim = np.shape(dataPerBin[0])[0]

if colDim != (HemiBound*2):
    temp_dataPerBin = {} # write over original data in temp

    print(f'1F1 data is included, now removing this data...')

    for i in range(int(len(NumBins))-1):
        dOut = dataPerBin[i][1:,:,:]
        print(f'Data out dims: {dOut.shape}')
        temp_dataPerBin[i] = dOut
        print(f'All Done! :)')
    print(f'Data shapes are now good!')
    
    dataPerBin = temp_dataPerBin # rewrite over old data

else:
    print(f'Data is correct dims!')

#### F1 and F2 data are stacked on top of each other so here we split them :)

In [ ]:
xF1Bins = {} # data is : 2f1 4f1 6f1 8f1
xF2Bins = {} # data is  2f2 4f2 6f2 8f2

for i in range(Bins):
    data2split = dataPerBin[i] # 8 x 128 x 78
    xF1Bins[i] = np.array(data2split[:HemiBound,:,:]) # harmonics x channels x trials  = 4 x 128 x 78
    xF2Bins[i] = np.array(data2split[HemiBound:,:,:]) # harmonics x channels x trials 

In [ ]:
pickBin = 0
pickTrial = 60
print(f'Channel activity per trial: harmonic, bin and hemifield')
ch_list = np.arange(0,NumChans,1)
for i in range(4):
    plt.scatter(ch_list,xF1Bins[pickBin][i,:,pickTrial], label = f'{(i+1)*2}F1') # plot per channel actiivity 
    plt.legend()
plt.show()

for i in range(4):
    plt.scatter(ch_list,xF2Bins[pickBin][i,:,pickTrial], label = f'{(i+1)*2}F2') # plot per channel actiivity 
    plt.legend()
plt.show()

### seperate data into pre and post 

In [ ]:
prepost_bound= int(NumTrials/2) # reshape will occure at this ind 

xF1_pre_post = {} # F1 pre post data
xF2_pre_post = {} # F2 pre post data

for i in range(Bins):
    xF1_pre_post[i] = np.reshape(xF1Bins[i], newshape = (4,NumChans,prepost_bound,2))
    xF2_pre_post[i] = np.reshape(xF2Bins[i], newshape = (4,NumChans,prepost_bound,2)) # 4 x 128 x pre / post trials  x 2# 

#### visualize data not for analysis - data is not comperable but more to make sure there is variance between trials, hemifields and harmonics

In [ ]:
pickBin = 0
pickTrial = 9
print(f'channel activity per trial: harmonic, bin and hemifield')
ch_list = np.arange(0,NumChans,1)

for i in range(4):
    plt.scatter(ch_list,xF1_pre_post[pickBin][i,:,pickTrial,0], label = f'{(i+1)*2}F1 pre') # plot per channel actiivity 
    plt.legend()
plt.show()
for i in range(4):
    plt.scatter(ch_list,xF1_pre_post[pickBin][i,:,pickTrial,1], label = f'{(i+1)*2}F1 post') # plot per channel actiivity 
    plt.legend()
plt.show()

for i in range(4):
    plt.scatter(ch_list,xF2_pre_post[pickBin][i,:,pickTrial,0], label = f'{(i+1)*2}F2 pre') # plot per channel actiivity 
    plt.legend()
plt.show()
for i in range(4):
    plt.scatter(ch_list,xF2_pre_post[pickBin][i,:,pickTrial,1], label = f'{(i+1)*2}F2 post') # plot per channel actiivity 
    plt.vlines(40, ymin = 0 , ymax = 3, color = 'black')
    plt.vlines(100, ymin = 0, ymax = 3, color = 'black')
    plt.legend()
plt.show()

#### Concate Data Across Bins to Get Average Activity Per Channel, Bin and Harmonic

In [ ]:
xF1_CleanAvgs = {}
xF2_CleanAvgs = {}

for prepost in range(2):
    xF1_CleanAvgs[prepost] = {} # make a new dict key for pre and then post 
    xF2_CleanAvgs[prepost] = {}
    for bin in range(Bins):
        #enter data dict
        xF1_dIn = xF1_pre_post[bin] # 4  x 128 x ntrials x pre / post
        xF2_dIn = xF2_pre_post[bin]
        # avg harmonic activity per channel
        xF1_CleanAvgs[prepost][bin] = np.nanmean(xF1_dIn[:,:,:,prepost],axis = -1) # average data per trial to get 1 val per channel
        xF2_CleanAvgs[prepost][bin] = np.nanmean(xF2_dIn[:,:,:,prepost],axis = -1) # average data per trial to get 1 val per channel

#### Sanity check plot

In [ ]:
pepo = ['pre', 'post']

binInd = 5
fig, axs = plt.subplots(1,2,figsize = (10,5),sharey = True)
for prepost in range(2):
    dIn = xF1_CleanAvgs[prepost]
    for hi in range(4):
        axs[prepost].scatter(ch_list,dIn[binInd][hi,:], label = f'{(hi+1)*2}F1 {pepo[prepost]}')
        axs[prepost].hlines(0,xmin = 0,xmax = 128, color = 'black')
    plt.suptitle('XF1 average channel activity per bin')
    axs[prepost].legend()

fig, axs = plt.subplots(1,2,figsize = (10,5),sharey = True)
for prepost in range(2):
    dIn = xF2_CleanAvgs[prepost]
    for hi in range(4):
        axs[prepost].scatter(ch_list,dIn[binInd][hi,:], label = f'{(hi+1)*2}F2 {pepo[prepost]}')
        axs[prepost].hlines(0,xmin = 0,xmax = 128, color = 'black')
    plt.suptitle('XF2 average channel activity per bin')
    axs[prepost].legend()


#### Combine Data Across Harmonics 

In [ ]:
XF1_Data = {}
XF2_Data = {}

for prepost in range(2):
    
    XF1_Data[prepost] = {}
    XF2_Data[prepost] = {}
    temp_XF1 = np.concatenate([xF1_CleanAvgs[prepost][x] for x in xF1_CleanAvgs[prepost]])
    temp_XF2 = np.concatenate([xF2_CleanAvgs[prepost][x] for x in xF2_CleanAvgs[prepost]])
    XF1_Data[prepost] = np.reshape(temp_XF1, newshape = (4,Bins,NumChans))
    XF2_Data[prepost] = np.reshape(temp_XF2, newshape =(4,Bins,NumChans))

In [ ]:
fig,axs = plt.subplots(1,2, figsize = (12,6), sharey = True)
plt.suptitle('Pre  + Post Channel Activity XF2')
for pltpos in range(2):
    pepo = XF2_Data[pltpos] # plot pre and post data
    for i in range(Bins):
        harmtwo  = pepo[0,i,:]
        ht = np.nanmean(harmtwo,axis = 0) # 128 points of avg channel activity pre or post 
        axs[pltpos].scatter(ch_list,harmtwo, label = f'Bin: {i}')
        axs[pltpos].legend()
# plt.title(f'activity per channel across all bins')

# Compute Difference in Response Activity Post - Pre Induction
## Method # 1: Post - Pre

### XF1 Data  

In [ ]:
f1_temp_pre = XF1_Data[0] # 4 x 6 x 128
f1_temp_post = XF1_Data[1] # 4 x 6 x 128

XF1_diffs = np.zeros_like((f1_temp_pre)) # temporary store of data 

for hi in range(4):
    for bin in range(Bins):
        diff = (f1_temp_post[hi,bin,:] - f1_temp_pre[hi,bin,:])
        XF1_diffs[hi,bin,:] = diff

### XF2 Data

In [ ]:
f2_temp_pre = XF2_Data[0] # 4 x 6 x 128
f2_temp_post = XF2_Data[1] # 4 x 6 x 128

XF2_diffs = np.zeros_like((f2_temp_pre)) # temporary store of data 

for hi in range(4):
    for bin in range(Bins):
        diff = (f2_temp_post[hi,bin,:] - f2_temp_pre[hi,bin,:])
        XF2_diffs[hi,bin,:] = diff

## **Checkpoint #2** Before Plotting Topos

In [ ]:
if np.sum(XF1_diffs == XF2_diffs) == 0:
    print(f'Data does not overlap, good sign') # check if data is the same
else:
    print(f'check data indcies as data is the same for both hemifields')

### Select Harmonic Data to Visualize, transpose data and Add Dead Channel for Montage

In [ ]:
Harmonic = 0 # pick harmoic data to plot on topo

### XF1 Difference Data 

In [ ]:
F1_Diffs = XF1_diffs[Harmonic,:,:].T  # 128 x 6 - 0 -== 2f
print(F1_Diffs.shape)

### XF2 Difference Data 

In [ ]:
F2_Diffs = XF2_diffs[Harmonic,:,:].T  # 128 x 6 - 0 -== 2f
print(F2_Diffs.shape)

# Select EEG Montage for MNE

## Import and Set EEG Sensor Location File and Set as Default Montage

In [ ]:
# eegMontageDir = 'C:\\plimon\\LTP_analysis\\eegSensorLocs\\9_18AverageNet128_v1.sfp'#AdultAverageNet128_v1.sfp'
# # Check if the file exists
# if os.path.exists(eegMontageDir):
#     # Load the electrode locations file
#     montageIn = mne.channels.read_custom_montage(eegMontageDir) # import file
#     print("Electrode locations loaded successfully.")
# else:
#     print("The specified file does not exist.")
# print(montageIn)
# #### Get channel names to use
# info = mne.create_info(ch_names=montageIn.ch_names, sfreq=1, ch_types="eeg")
# info.set_montage(montageIn)

In [ ]:
montageIn = mne.channels.make_standard_montage("GSN-HydroCel-128")
info = mne.create_info(ch_names=montageIn.ch_names, sfreq=1, ch_types="eeg")
info.set_montage(montageIn)
print(montageIn)

## Visualize Data

In [ ]:
p = XF1_Data[0]
data = p[Harmonic,:,:].T  # 128 x 6 - 0 -== 2f


for i in range(Bins):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 10),gridspec_kw=dict(height_ratios=[3]))
    axes.set_title(f' Pre F1 Bin # {i+1}', fontsize = 30)
    axes.axis('off')

    dataIn = data[:,i] # plot data per bin
    # hashtag below to plot all channel activity
    dataIn[~ocm] = 0 # want to index good channels only and set the rest channels to 0 value
    im,_ = plot_topomap(dataIn, info,mask = ocm, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 4)
    divider = make_axes_locatable(axes)
    cax = divider.append_axes('right', size='5%', pad=0.2)
    cbar = plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
p = XF1_Data[1]
data = p[Harmonic,:,:].T  # 128 x 6 - 0 -== 2f


for i in range(Bins):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 12),gridspec_kw=dict(height_ratios=[3]))
    axes.set_title(f'Post F1 Bin # {i+1}', fontsize = 30)
    axes.axis('off')

    dataIn = data[:,i]
    dataIn[~ocm] = 0 # want to index good channels only and set the rest channels to 0 value
    im,_ = plot_topomap(dataIn, info,mask = ocm, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 3)
    divider = make_axes_locatable(axes)
    cax = divider.append_axes('right', size='5%', pad=0.2)
    cbar = plt.colorbar(im, cax=cax)
    plt.show()

#### Plot F1 Difference post induction

In [ ]:
for i in range(Bins):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 12),gridspec_kw=dict(height_ratios=[3]))
    axes.set_title(f'F1 response change post induction, Bin # {i+1}', fontsize = 30)
    axes.axis('off')

    dataIn = F1_Diffs[:,i]
    dataIn[~ocm] = 0 # want to index good channels only and set the rest channels to 0 value
    im,_ = plot_topomap(dataIn, info,mask = ocm, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 3)
    divider = make_axes_locatable(axes)
    cax = divider.append_axes('right', size='5%', pad=0.2)
    cbar = plt.colorbar(im, cax=cax)
    plt.show()

##### Ploting F2 Data Now, 2nd harmonic

In [ ]:
p = XF2_Data[0]

data = p[0,:,:].T  # 128 x 6 - 0 -== 2f

for i in range(Bins):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 12),gridspec_kw=dict(height_ratios=[3]))
    axes.set_title(f'Pre F2 Bin # {i+1}', fontsize = 30)
    axes.axis('off')

    dataIn = data[:,i] # 129 array
    dataIn[~ocm] = 0 # want to index good channels only and set the rest channels to 0 value
    im,_ = plot_topomap(dataIn, info,mask = ocm, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 3)
    divider = make_axes_locatable(axes)
    cax = divider.append_axes('right', size='5%', pad=0.2)
    cbar = plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
p = XF2_Data[1]
data = p[0,:,:].T  # 128 x 6 - 0 -== 2f

for i in range(Bins):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 12),gridspec_kw=dict(height_ratios=[3]))
    axes.set_title(f'Post F2 Bin # {i+1}', fontsize = 30)
    axes.axis('off')

    dataIn = data[:,i]
    dataIn[~ocm] = 0 # want to index good channels only and set the rest channels to 0 value
    im,_ = plot_topomap(dataIn, info,mask = ocm, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 4)
    divider = make_axes_locatable(axes)
    cax = divider.append_axes('right', size='5%', pad=0.2)
    cbar = plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
for i in range(Bins):
    #plt.plot(topodata[:,i])
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 12),gridspec_kw=dict(height_ratios=[3]))
    axes.set_title(f'F2 response change post induction, Bin # {i+1}', fontsize = 30)
    #placeholder = np.random.random((10, 10))
    #axs.imshow(placeholder)
    axes.axis('off')

    dataIn = F2_Diffs[:,i]
    dataIn[~ocm] = 0 # want to index good channels only and set the rest channels to 0 value
    im,_ = plot_topomap(dataIn, info,mask = ocm, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 3)
    divider = make_axes_locatable(axes)
    cax = divider.append_axes('right', size='5%', pad=0.2)
    cbar = plt.colorbar(im, cax=cax)
    plt.show()

In [ ]:
# topodata = p[0,:,:].T
# print(topodata.shape)
# emptyChData = np.zeros((1,6))
# print(emptyChData.shape)

# dataFull = np.array(np.concatenate((topodata,emptyChData)))

# data = dataFull[:,4]

# # create array with 4 points for our 4 channels
# # in the same order as provided in ch_names
# fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10),gridspec_kw=dict(height_ratios=[3]))
# axes.set_title(f'subj topo plot', fontsize = 30)
# # placeholder = np.random.random((10, 10))
# # axes.imshow(placeholder)
# axes.axis('off')

# #info = create_info(ch_names=montageIn._names, sfreq=1, ch_types='eeg')
# # channel names I provided are part of a standard montage
# #info.set_montage(montageIn) # import file)
# im,_ = plot_topomap(data, info, axes = axes, show =False,cmap = 'Spectral_r', res = 32, contours = 8)
# divider = make_axes_locatable(axes)
# cax = divider.append_axes('right', size='5%', pad=0.2)
# cbar = plt.colorbar(im, cax=cax)
# plt.show()

### Good confined example of what data formate is supposed to look like for mne plotting success

In [ ]:
# from mne import create_info
# from mne.viz import plot_topomap
# import numpy as np

# # create array with 4 points for our 4 channels
# # in the same order as provided in ch_names
# data = np.random.randn(4)  
# info = create_info(ch_names=['CPz', 'Oz', 'POz', 'Fz'], sfreq=1000, ch_types='eeg')
# # channel names I provided are part of a standard montage
# info.set_montage('standard_1020')

# plot_topomap(data, info)

In [ ]:
# topodata = p[0,:,:].T  # 128 x 6 - 0 -== 2f
# #have to add some fck channel data because egi file included one extra channel
# emptyChData = np.zeros((1,6)) # 1 x 6
# data = np.array(np.concatenate((topodata,emptyChData))) # 129 x 6 


# for i in range(6):
#     #plt.plot(topodata[:,i])
#     fig, axs = plt.subplots(figsize = (6,6))
#     axs.set_title(f'Bin # {i}')
#     axs.legend()
#     print(i)
#     dataIn = data[:,i]
#     mne.viz.plot_topomap(dataIn, info, axes = axs)

#### plot formats of the past and present(?)

In [ ]:
# fig = plt.figure(figsize = (10,10))
# ax = fig.add_subplot(111, projection='3d')
# montageIn.plot(kind='3d', axes=ax)
# ax.set_title('Sensor Locations')
# plt.show()
# # Plot the sensors on a 2D head model
# fig, ax = plt.subplots()
# montageIn.plot(kind='topomap', show_names=True, axes=ax)
# ax.set_title('Sensor Locations (2D)')
# plt.show()

In [ ]:
# #montge default info in case egi outut file fails me 
# montage_type = mne.channels.make_standard_montage("GSN-HydroCel-128")
# print(montage_type)

In [ ]:
# h_label = ['2F','4F','6F','8F']

# fig,axs = plt.subplots(6,1, figsize = (10,40))
# for i in range(6):
#     dIn = axs[i].imshow(XF1_Data[0][:,i,ExportChans],aspect='auto', cmap=sns.cubehelix_palette(as_cmap=True))
#     axs[i].set_xticks(np.arange(len(ExportChans)))
#     axs[i].set_xticklabels(ExportChans,rotation=45)
#     axs[i].set_yticks(np.arange(len(h_label)))  # Set y-ticks to match the length of y_labels
#     axs[i].set_yticklabels(h_label) 
#     axs[i].set_title('Pre F1 Data')
#     axs[i].set_xlabel('Channels')
#     axs[i].set_ylabel('Amplitude')
#plt.show()
# fig,axs = plt.subplots(6,1, figsize = (10,40))
# for i in range(6):
#     dIn = axs[i].imshow(XF1_Data[1][:,i,ExportChans],aspect='auto', cmap=sns.cubehelix_palette(as_cmap=True))
#     axs[i].set_xticks(np.arange(len(ExportChans)))
#     axs[i].set_xticklabels(ExportChans,rotation=45)
#     axs[i].set_yticks(np.arange(len(h_label)))  # Set y-ticks to match the length of y_labels
#     axs[i].set_yticklabels(h_label) 
#     axs[i].set_title('Post F1 Data')
#     axs[i].set_xlabel('Channels')
#     axs[i].set_ylabel('Amplitude')
# fig,axs = plt.subplots(6,1, figsize = (10,40))
# for i in range(6):
#     dIn = axs[i].imshow(XF2_Data[0][:,i,ExportChans],aspect='auto', cmap=sns.cubehelix_palette(as_cmap=True))
#     axs[i].set_xticks(np.arange(len(ExportChans)))
#     axs[i].set_xticklabels(ExportChans,rotation=45)
#     axs[i].set_yticks(np.arange(len(h_label)))  # Set y-ticks to match the length of y_labels
#     axs[i].set_yticklabels(h_label) 
#     axs[i].set_title('Pre F2 Data')
#     axs[i].set_xlabel('Channels')
#     axs[i].set_ylabel('Amplitude')
# fig,axs = plt.subplots(6,1, figsize = (10,40))
# for i in range(6):
#     dIn = axs[i].imshow(XF2_Data[1][:,i,ExportChans],aspect='auto', cmap=sns.cubehelix_palette(as_cmap=True))
#     axs[i].set_xticks(np.arange(len(ExportChans)))
#     axs[i].set_xticklabels(ExportChans,rotation=45)
#     axs[i].set_yticks(np.arange(len(h_label)))  # Set y-ticks to match the length of y_labels
#     axs[i].set_yticklabels(h_label) 
#     axs[i].set_title('Post F2 Data')
#     axs[i].set_xlabel('Channels')
#     axs[i].set_ylabel('Amplitude')